In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, math
!date

Tue Apr 30 10:13:30 AM PDT 2024


# Generate full-USA simulated decennial census with only 64GB of memory, using Dask

In [2]:
import pseudopeople as psp
psp.__version__

'1.0.1.dev14+g217800e'

In [3]:
import dask
from dask.distributed import LocalCluster

In [4]:
full_data_path = '/mnt/team/simulation_science/pub/models/vivarium_census_prl_synth_pop/results/release_02_yellow/full_data/united_states_of_america/2023_08_21_16_35_27/final_results/2023_08_31_15_58_01/pseudopeople_simulated_population_usa_2_0_0/'
# !ls -haltd $full_data_path 

In [5]:
import time

In [6]:
%%time

import dask
dask.config.set({"distributed.worker.memory.terminate": False})
dask.config.set({"distributed.comm.retry.count": 5})
dask.config.set({"distributed.comm.timeouts.connect": 5 * 60})
dask.config.set({"distributed.comm.timeouts.tcp": 5 * 60})

cluster = LocalCluster(n_workers=10, threads_per_worker=1, memory_limit=f"6GB")
client = cluster.get_client()
display(client)
df = psp.generate_decennial_census(
    source=full_data_path,
    engine='dask')
dask.distributed.wait(df.persist())
#         df.compute()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 10,Total memory: 55.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33769,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 55.88 GiB
Comm: tcp://127.0.0.1:36471,Total threads: 1
Dashboard: http://127.0.0.1:44837/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:40541,


Applying noise:   7%|▋         | 1/15 [00:22<05:17, 22.69s/type]2024-04-30 10:31:48,504 - distributed.worker.memory - WARNING - Worker is at 93% memory usage. Pausing worker.  Process memory: 5.22 GiB -- Worker memory limit: 5.59 GiB
2024-04-30 10:31:52,554 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.44 GiB -- Worker memory limit: 5.59 GiB
Applying noise:   7%|▋         | 1/15 [00:36<08:35, 36.79s/type]2024-04-30 10:32:01,158 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.71 GiB -- Worker memory limit: 5.59 

CPU times: user 13min 4s, sys: 5min 55s, total: 19min
Wall time: 1h 41min 19s


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 282)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 144)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 257)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 135)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 248)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 262)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 1)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c593490d1', 249)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('lambda-3ead62ed3b546b22f10b3a4c59

In [7]:
df.head(n=100)

Applying noise:   0%|          | 0/15 [00:00<?, ?type/s]2024-04-30 11:55:29,965 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.59 GiB -- Worker memory limit: 5.59 GiB


,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,28_0,28_6,Doris,J,Jackson,70,01/28/1950,2504,martinho ave,NaN,pflugerville,TX,77076,Household,Reference person,Female,Black,2020
1,28_1,28_7,Eric,J,Perry,54,04/01/1965,1003,ne olive way,NaN,madsen,WI,54466,Household,Reference person,Male,White,2020
2,28_2,28_8,J,J,Perez,41,06/27/1978,3624,arch st,NaN,sparks,NV,89121,Household,Reference person,Male,Latino,2020
3,28_3,28_8,Diane,J,Perez,38,09/19/1981,3624,arch st,NaN,sparks,NV,89121,Household,Opposite-sex spouse,Female,Latino,2020
4,28_4,28_8,Daniel,J,Perez,15,06/04/2004,3624,arch st,NaN,NaN,NV,89121,Household,Biological child,Male,Latino,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,28_97,28_43,Deja,J,Elvir,14,11/08/2005,5831,nth avenue,NaN,kempner,TX,75070,Household,Biological child,Female,White,2020
97,28_98,28_44,Edith,J,Cousins,71,10/09/1948,7324,e center st,NaN,nocona,TX,77389,Household,Reference person,Female,Black,2020
98,28_99,28_45,Frederick,J,Cowgill,75,09/23/1944,270,19th st,NaN,hartland,WI,53406,Household,Reference person,Male,White,2020
99,28_100,28_45,Margaret,L,Cowgill,70,12/30/1949,270,19th st,NaN,hartland,WI,53406,Household,Opposite-sex spouse,NaN,White,2020
